# 关系数据库

基础的SQL学习

## Python DB-API

通过API包，使用Python控制数据库。

使用的模式大致如下：

先链接数据库（connect），这样就能得到一个Connection的实例，该实例可以：
- `.cursor()`，生成Cursor实例，即光标，可以进一步：
    - `.execute(SQL Code)`，执行SQL代码，然后
    - `.fetchone()`，获得第一行结果，或者
    - `.fetchall()`，获得所有结果
- 当执行一些修改代码（如插入、删除等），需要执行`.commit()`确认更改，或者
- 执行`.rollback()`回滚修改操作。

### SQL注入攻击

当我们在python中执行一些SQL查询时，可能会写如下的代码：
```python
c = connection.cursor()
c.execute("insert into posts values ('%s')" % content)
```

如上代码在执行普通查询时，并不会产生错误，但是如果插入的`content`中，包含`'`，就会报错

当插入的content为`'); delete from posts; --`时，会导致我们数据库`posts`中的全部数据被删除，这就是**SQL注入攻击**

我们可以修改代码为：
```python
c.execute("insert into posts values(%s)",(content,))
```
来避免SQL注入攻击。

**关键：在使用Python DB-API时，执行SQL代码不要使用输出格式化，如%，format或者f''string等**

### 脚本注入攻击

如果将content修改为：
```javascript
<script>
setTimeout(function() {
    var tt = document.getElementById('content');
    tt.value = "<h2 style='color: #FF6699; font-family: Comic Sans MS'>Spam, spam, spam, spam,<br>Wonderful spam, glorious spam!</h2>";
    tt.form.submit();
}, 2500);
</script>
```
也会发生错误，浏览器会不断输出spam（如上js代码的输出），这是因为，对于SQL来说，如上内容即字符串，但是当返回到浏览器中，进行解析时，浏览器会把它当作js并执行。

这并不是我们想要的结果，那该如何解决呢？

我们可以使用[Bleach](https://bleach.readthedocs.io/en/latest/)库，来帮我们对输入内容进行清理：
```python
# 安装bleach
pip install bleach
# 使用
import bleach

bleach.clean('an <script>evil()</script> example')
```
输出：`u'an &lt;script&gt;evil()&lt;/script&gt; example'`


如何处理数据库中已经存在的垃圾信息呢？
- 使用`UPDATE`，将垃圾信息替换为无害信息，比如说cheese!
    ```SQL
    UPDATE table
    SET column = value
    WHERE xxxxx;
    ```
- 使用`DELETE`，将垃圾信息删除
    ```SQL
    DELETE FROM table
    WHERE xxxxx;
    ```

## 数据库设计

要遵循规范式设计（Normalized Design）：
- 每一行都要有相同数量的列；
- 每张表中包含关键列（key），每一行都说明了该关键列的某些信息；
- 每张表中不需要包含说明非关键列信息的列，   
    比如：`姓名，年龄，购买商品，商品价格`中，`姓名`为关键列，而`商品价格`是对`购买商品`的说明，  
    所以，我们需要拆成两张表，即`姓名，年龄，购买商品`与`商品，商品价格`
- 表中的各列不应该有关系暗示，会产生误导，  
    比如：在员工技能表中
    <img src="https://s3.ax1x.com/2020/12/03/DTVuuR.png" alt="DTVuuR.png" border="0" width="300px"/>
    
    可能会误导，Annabel会Databases与English有关；会Linux与French有关。这时候，我们最好可以拆成两张表：
    
    <img src="https://s3.ax1x.com/2020/12/03/DTVRrn.md.png" alt="DTVRrn.png" border="0" width="500px"/>
    


# 后端基础
## CRUD（Create，Read，Update，Delete）模式

先来了解一下**对象关系映射（Object Relational Mapping,ORM）**，是一种程序设计技术，用于实现面向对象编程语言里不同类型系统的数据之间的转换。其实就是以对象形式表示的数据库，更方便我们在面向对象语言（如Python）中使用。

在Python中，我们使用最多的工具是[SQLAlchemy](https://www.sqlalchemy.org/)

### 使用SQLAlchemy创建数据库
#### 配置代码
用于导入所有必要的模块

`database_setup.py`
```python
import sys
from sql

from sqlalchemy import Column, ForeignKey, Integer, String
# 导入declarative_base，用于配置和类代码
from sqlalchemy.ext.declarative import declarative_base
# 用于创建外键关系，也会在创建mapper上派上用场
from sqlalchemy.orm import relationship
# 用于文末代码（创建或链接数据库）
from sqlalchemy import create_engine

Base = declarative_base()

# 连接数据库，这里也可以链接其他数据库，比如mysq等
engine = create_engine('sqlite:///restaurantmenu.db')

# 进入数据库
Base.metadata.create_all(engine)
```

#### Class 类
使用Python中的类代表我们的数据库

比如说，我们分别创建两张表：
- 餐厅表
- 菜单表

```python
class Restaurant(Base):
    
    
class MenuItem(Base):

```

#### Table 表
代表数据库中特定的表格

在如上对应的类中，添加表名（建议小写）
```python
# 在Restaurant类中
__tablename__ = 'restaurant'


# 在MenuItem类中
__tablename__ = 'menu_item'
```

#### Mapper映射器
用于将表格中的列与代表它的类链接，比如
```python
String(250) # 创建一个长度最多为250的字符串
Integer # 创建整型
relationship(Class) # 该表与其他表之间的关系
nullable = False # 是否允许有空值
primary_key = True # 是否是主键
ForeignKey('some_table.id') #与其他表中某列的外键关系
```

在刚才创建的表格中，分别添加上列：
- 餐厅表：
    - 餐厅名称
    - 餐厅id（主键）
- 菜单表：
    - 菜单名称
    - 菜单id
    - 菜品名称
    - 菜品描述
    - 菜品价格
    - 餐厅id
    

```python
# 在Restaurant类中
name = Column(String(80),nullable=False)# 餐厅名称，字符类型，不允许为空
id = Column(Integer,primary_key=True)
    


# 在MenuItem类中
name = Column(String(80),nullable=False)
id = Column(Integer,primary_key=True)
course = Column(String(250))
description = Column(String(250))
price = Column(String(8))
restaurant_id = Column(Integer,ForeignKey('restaurant.id'))
restaurant = relationship(Restaurant)
```

#### 合并所有代码

将如上代码合并，代码架构可以如下所示：
![DT4MOP.png](https://s3.ax1x.com/2020/12/03/DT4MOP.png)

In [3]:
import os
import sys
from sqlalchemy import Column, ForeignKey, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship
from sqlalchemy import create_engine

Base = declarative_base()


class Restaurant(Base):
    __tablename__ = 'restaurant'

    id = Column(Integer, primary_key=True)
    name = Column(String(250), nullable=False)


class MenuItem(Base):
    __tablename__ = 'menu_item'

    name = Column(String(80), nullable=False)
    id = Column(Integer, primary_key=True)
    description = Column(String(250))
    price = Column(String(8))
    course = Column(String(250))
    restaurant_id = Column(Integer, ForeignKey('restaurant.id'))
    restaurant = relationship(Restaurant)


engine = create_engine('sqlite:///restaurantmenu.db')
# 创建
Base.metadata.create_all(engine)

### CRUD-CREATE

现在我们已经有了一个空的数据库，现在创建一些数据

In [5]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from database_setup import Base, Restaurant, MenuItem

In [6]:
# 链接数据库
engine = create_engine('sqlite:///restaurantmenu.db')

In [7]:
# 用bind将engine与Base绑定在一起
Base.metadata.bind = engine

In [9]:
# 创建一个session，与数据库进行会话
DBsession = sessionmaker(bind=engine)
# 实例化session
# 我们可以使用session进行CRUD操作，但之后需要使用commit进行确认
session = DBsession()

添加数据时，可以按照如下模板：
```python 
newEntry = ClassName(property="value",...)
session.add(newEntry)
session.commit()
```

In [10]:
# 添加餐厅
myFirstRestaurant = Restaurant(name="Pizza Palace")
session.add(myFirstRestaurant)
session.commit()

In [11]:
# 查看是否添加成功
session.query(Restaurant).all()

可以发现，确实有数据存在内存的某处了。至于如何筛选，我们后面再了解。

In [12]:
# 添加菜单
cheesepizza = MenuItem(name="Cheese Pizza", description="Made by Franchise",
                      course="Entree",price="$8.99",restaurant=myFirstRestaurant)
session.add(cheesepizza)
session.commit()

In [13]:
# 查看是否添加成功
session.query(MenuItem).all()

### CRUD-READ

In [14]:
# 查看第一条
firstResult = session.query(Restaurant).first()

In [15]:
# 使用类调用的方式去查看
firstResult.name

'Pizza Palace'

当有多条时，只需要使用for循环配合就可以，如：
```python
allResult = session.query(Restaurant).all()
for item in allResult:
    print(item.name)
```

### CRUD-QUERY（查询）

我们还需要涉及到一些条件筛选，就像在SQL中做的那样。

In [16]:
# 首先，我们需要使用query创建一个query对象
q = session.query(Restaurant)

当然，也可以针对某几列，如：
```python
q = session.query(MenuItem.name,MenuItem.description)
```

#### filter

- 方案A：使用Pythonic的方式

**算术运算符**：

```python
q.filter(User.name == 'ed') # !=不等于；>大于；<小于等等
```


**成员运算符：**
```python
# IN
q.filter(User.name.in_(['a','b','v']))
# NOT IN
q.filter(~User.name.in_(['a','b','v']))
```

**模糊筛选：**
```python
q.filter(User.name.like('%ed%'))
```

**NULL筛选**
```
# IS NULL
query.filter(User.name == None)
# 或者
query.filter(User.name.is_(None))

# NOT NULL
query.filter(User.name != None)
# 或者
query.filter(User.name.isnot(None))
```

**多条件逻辑运算符**
```python
# 与
# 方法1
from sqlalchemy import and_
query.filter(and_(User.name == 'ed', User.fullname == 'Ed Jones'))

# 方法2
query.filter(User.name == 'ed', User.fullname == 'Ed Jones')

# 方法3
query.filter(User.name == 'ed').filter(User.fullname == 'Ed Jones')
```

```python
# 或
from sqlalchemy import or_
query.filter(or_(User.name == 'ed', User.name == 'wendy'))
```

```python
# 非
q.filter(~User.name.in_(['a','b','v']))
```

- 方案B：使用SQL原生语句

```python
from sqlalchemy import text
q.filter(text("id<224"))
# 或者
q.from_statement(text("SELECT * FROM users where name=:name")).params(name='ed')
```


#### 聚合，排序，联合表

**聚合**
```SQL
SELECT school, COUNT(*) AS c 
FROM persons 
WHERE gender="male" 
GROUP BY age 
HAVING c >1
```
等价于
```python
from sqlalchemy import func
# label用于别名，与SQL代码中的AS一致
# 对于其他的聚合计算函数，如SUM，AVG等分别对应于
nums = func.count('*').label('c')

results = sessin.query( Person.school, nums ).filter(
    Person.gender=='male'
).group_by(
    Person.age
).having(
    nums > 10
)
```

**排序**
```python
q.order_by(User.create_time.desc())
# 或者
from sqlalchemy import desc
q.order_by(desc(Usser_ID))
# 或者直接在定义数据类（Class）的时候，添加上默认排序方式，如
class User(Base):　　
　　__tablename__ = "user"　　
　　id = Column(Integer , primary_key=True , autoincrement=True)　　
　　name = Column(String(50) , nullable=False)　　
　　create_time = Column(DateTime , nullable=False , default=datetime.now)
　　
　　__mapper_args__ = {　　
　　"order_by":create_time.desc()　　
　　}　　

# 这样，所有的筛选都会按照create_time的倒序给出结果

```


**多表联合**
```python
# User与Address为表
session.query(User, Address).\
    filter(User.id==Address.user_id).\
    filter(Address.email_address=='jack@google.com')
# 等价于
# 此方法需要在定义数据库时，设定好外键
session.query(User).join(Address).\
    filter(Address.email_address=='jack@google.com')
# 如果没有设置好外键，需要指定关键列
session.query(User).\
    join(Address, User.id==Address.user_id).\
    filter(Address.email_address=='jack@google.com')
# 如上的join默认为inner-join
# outerjoin的话直接使用函数outerjoin即可，不过这里都是left outerjoin
# 如果想使用full outerjoin的话，只需要创建两个left outerjoin，然后再合并即可，如下所示：
q1 = (db.session.query(
        tb1.user_id.label('u_id'),
        func.count(tb1.id).label('tb1_c')
    )
    .group_by(tb1.user_id)
).cte('q1')

q2 = (db.session.query(
        tb2.user_id.label('u_id'),
        func.count(tb2.id).label('tb2_c')
    )
    .group_by(tb2.user_id)
).cte('q2')

result = db.session.query(
    func.coalesce(q1.u_id, q2.u_id).label('u_id'),
    q1.tb1_c,
    q2.tb2_c
).join(
    q2,
    q1.u_id == q2.u_id,
    full=True
)
```

更多筛选可以去这里查询：[SQLAlchemy-Query](https://www.osgeo.cn/sqlalchemy/orm/query.html)

### CRUD-UPDATE

```python
# 先筛选对应的数据，并用one返回
need_update = session.query(MenuItem).filter(id=8).one()
# 更新某数据
need_update.price = "$2.99"
session.add(need_update)
session.commit()
```


### CRUD-DELETE

```python
# 先筛选对应的数据
need_delete = session.query(MenuItem).filter(id=8).one()
# 删除
session.delete(need_delete)
session.commit()
```

## 搭建web服务 